We set 'Not assigned' values as Nan for easier data handling. The above code replaces the Not assigned values as NaN
Note: We are using pandas df and not beautifulsoup as per option mentioned in the instructions for this project.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
toronto_table = pd.read_csv('scraped_table.tsv', sep='\t', na_values='Not assigned')
#In the line above we are changing all \'Not assigned\' values as NaN
toronto_table.dropna(subset=['Borough'],inplace=True)
print(toronto_table.shape)

(210, 3)


### In order to perform groupby and agg function we are converting all data types to string for uniformity.
   We then do groupby-agg and join the returned list from groupby object using a comma

In [3]:
toronto_table['Neighbourhood']  = toronto_table['Neighbourhood'].astype(str)
toronto_table['Postcode'] = toronto_table['Postcode'].astype(str)
toronto_table['Borough'] = toronto_table['Borough'].astype(str)
'''In order to perform groupby and agg function we are converting all data tpes to string for uniformity.
   We then do groupby-agg and join the returned list from groupby object using a comma '''
toronto_table = toronto_table.groupby(['Postcode','Borough'])['Neighbourhood'].agg(lambda col: ','.join(col)).reset_index()
print(toronto_table)

    Postcode      Borough                                      Neighbourhood
0        M1B  Scarborough                                      Rouge,Malvern
1        M1C  Scarborough               Highland Creek,Rouge Hill,Port Union
2        M1E  Scarborough                    Guildwood,Morningside,West Hill
3        M1G  Scarborough                                             Woburn
4        M1H  Scarborough                                          Cedarbrae
..       ...          ...                                                ...
98       M9N         York                                             Weston
99       M9P    Etobicoke                                          Westmount
100      M9R    Etobicoke  Kingsview Village,Martin Grove Gardens,Richvie...
101      M9V    Etobicoke  Albion Gardens,Beaumond Heights,Humbergate,Jam...
102      M9W    Etobicoke                                          Northwest

[103 rows x 3 columns]


## NOTE: Due to page fitting issue, scroll down for  column 3 

In [4]:
print(toronto_table.tail(40))
print(toronto_table.shape)

    Postcode           Borough  \
63       M5N   Central Toronto   
64       M5P   Central Toronto   
65       M5R   Central Toronto   
66       M5S  Downtown Toronto   
67       M5T  Downtown Toronto   
68       M5V  Downtown Toronto   
69       M5W  Downtown Toronto   
70       M5X  Downtown Toronto   
71       M6A        North York   
72       M6B        North York   
73       M6C              York   
74       M6E              York   
75       M6G  Downtown Toronto   
76       M6H      West Toronto   
77       M6J      West Toronto   
78       M6K      West Toronto   
79       M6L        North York   
80       M6M              York   
81       M6N              York   
82       M6P      West Toronto   
83       M6R      West Toronto   
84       M6S      West Toronto   
85       M7A  Downtown Toronto   
86       M7R       Mississauga   
87       M7Y      East Toronto   
88       M8V         Etobicoke   
89       M8W         Etobicoke   
90       M8X         Etobicoke   
91       M8Y  

#### We replace the values of 'nan' (remember it is a string rather than a np.nan value we check for since we converted all the values in all fields as 'str')

In [5]:
toronto_table['Neighbourhood'].replace('nan', toronto_table['Borough'],inplace=True)
toronto_table.shape


(103, 3)

In [6]:
toronto_table.tail(11)

,Postcode,Borough,Neighbourhood
92,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw..."
93,M9A,Queen's Park,Queen's Park
94,M9B,Etobicoke,"Cloverdale,Islington,Martin Grove,Princess Gar..."
95,M9C,Etobicoke,"Bloordale Gardens,Eringate,Markland Wood,Old B..."
96,M9L,North York,Humber Summit
97,M9M,North York,"Emery,Humberlea"
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


In [7]:
toronto_table.shape

(103, 3)

In [8]:
lat_long = pd.read_csv('Geospatial_Coordinates.csv')
lat_long.tail(5)
lat_long.rename(columns={'Postal Code':'Postcode'},inplace=True)
lat_long

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [9]:
lat_long['Postcode'].dtypes

dtype('O')

## Part 2:
### Let us sort both the DFs by the column - 'Postal code'. Using inner join on the postcode to add Lat, Long to the table.

In [10]:
toronto_table.sort_values(by='Postcode') 
lat_long[['Postcode','Latitude','Longitude']].sort_values(by='Postcode')
toronto_table = pd.merge(toronto_table, lat_long, on=['Postcode'], how = 'inner')


In [11]:
toronto_table.head(40)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [12]:
# We are choosing just those Boroughs that have Toronto as its Borough
toronto_venues = toronto_table[toronto_table.Borough.str.contains('Toronto')]
toronto_venues


,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049


## Using K clusters of value '5'. Clustering based on the Latitude and Longitude of each location.

In [13]:
# import k-means from clustering stage based on the latitude and longitude
from sklearn.cluster import KMeans
# run k-means clustering
kclusters = 5
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_venues.drop(columns=['Postcode', 'Borough', 'Neighbourhood']) )
kmeans.labels_

array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       1, 1, 3, 3, 3, 4, 4, 4, 3, 2, 3, 3, 2, 2, 2, 4, 0])

In [14]:
toronto_merged = toronto_venues
toronto_merged['Cluster Labels'] = kmeans.labels_
toronto_merged

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,1
47,M4S,Central Toronto,Davisville,43.704324,-79.388790,1
48,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160,1
49,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049,1


### Analyzing Each Neighborhood based on 'Borough' in Toronto.

In [15]:
# map rendering library
import folium 

toronto_merged

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,1
47,M4S,Central Toronto,Davisville,43.704324,-79.388790,1
48,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160,1
49,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049,1


### Visualizing  the clusters using Folium

In [16]:
#!conda install -c conda-forge gmaps
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent = "foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       


## We see above the map of 5 clusters based on the type of Boroughs.

In [17]:
map_clusters